In [2]:
import numpy as np
import math 
from scipy.stats import norm
from scipy.stats import truncnorm
import scipy.stats as stats

In [3]:
def autoselect_scales_mix_norm(betahat, sebetahat, max_class=None, mult=2):
    sigmaamin = np.min(sebetahat) / 10
    if np.all(betahat**2 < sigmaamin**2):  # Fix the typo and ensure logical comparison
        sigmaamax = 8 * sigmaamin
    else:
        sigmaamax = 2*np.sqrt(np.max(betahat**2 - sebetahat**2))
    
    if mult == 0:
        out = np.array([0, sigmaamax / 2])
    else:
        npoint = math.ceil(math.log2(sigmaamax / sigmaamin) / math.log2(mult))

        # Generate the sequence (-npoint):0 using np.arange
        sequence = np.arange(-npoint, 1)

        # Calculate the output
        out = np.concatenate(([0], (1/mult) ** (-sequence) * sigmaamax))
        if max_class!=None:
            # Check if the length of out is equal to max_class
            if len(out) != max_class:
            # Generate a sequence from min(out) to max(out) with length max_class
                out = np.linspace(np.min(out), np.max(out), num=max_class)
        
    
    return out
     
def autoselect_scales_mix_exp(betahat, sebetahat, max_class=None , mult=1.5,tt=1.5):
    sigmaamin = np.min(sebetahat) / 10
    if np.all(betahat**2 < sigmaamin**2):  # Fix the typo and ensure logical comparison
        sigmaamax = 8 * sigmaamin
    else:
        sigmaamax = tt*np.sqrt(np.max(betahat**2  ))
    
    if mult == 0:
        out = np.array([0, sigmaamax / 2])
    else:
        npoint = math.ceil(math.log2(sigmaamax / sigmaamin) / math.log2(mult))

        # Generate the sequence (-npoint):0 using np.arange
        sequence = np.arange(-npoint, 1)

        # Calculate the output
        out = np.concatenate(([0], (1/mult) ** (-sequence) * sigmaamax))
        if max_class!=None:
            # Check if the length of out is equal to max_class
            if len(out) != max_class:
            # Generate a sequence from min(out) to max(out) with length max_class
                out = np.linspace(np.min(out), np.max(out), num=max_class)
                if(out[2] <1e-2 ):
                 out[2: ] <- out[2: ] +1e-2
         
    
    return out

    
class PosteriorMeanExp:
    def __init__(self, post_mean, post_mean2, post_sd):
        self.post_mean = post_mean
        self.post_mean2 = post_mean2
        self.post_sd = post_sd

def posterior_mean_exp(betahat, sebetahat, log_pi, scale):
    assignment = np.exp(log_pi)
    assignment = assignment / assignment.sum(axis=1, keepdims=True)
    mu = 0
    post_assign = np.zeros((betahat.shape[0], scale.shape[0]))
    
    for i in range(betahat.shape[0]):
        post_assign[i,] = wpost_exp(x=betahat[i],
                                    s=sebetahat[i], 
                                    w=assignment[i,],
                                    scale=scale) 
    
    post_mean = np.zeros(betahat.shape[0])
    post_mean2 = np.zeros(betahat.shape[0])

    for i in range(post_mean.shape[0]):
        post_mean[i] = sum(post_assign[i, 1:] * my_etruncnorm(0,
                                                              np.inf,
                                                              betahat[i] - sebetahat[i]**2 * (1/scale[1:]), 
                                                              sebetahat[i]))
        post_mean2[i] = sum(post_assign[i, 1:] * my_e2truncnorm(0,
                                                                99999, #some weird warning for inf so just use something large enough for b
                                                                betahat[i] - sebetahat[i]**2 * (1/scale[1:]), 
                                                                sebetahat[i]))
        post_mean2[i] = max(post_mean[i], post_mean2[i])
    
    if np.any(np.isinf(sebetahat)):
        inf_indices = np.isinf(sebetahat)
        a = 1/scale[1:]
        # Equivalent of `post$mean[is.infinite(s)]` 
        post_mean[inf_indices] = np.sum(post_assign[inf_indices, 1:] / a, axis=1)

        # Equivalent of `post$mean2[is.infinite(s)]`
        post_mean2[inf_indices] = np.sum(2 * post_assign[inf_indices, 1:] / a**2, axis=1)

    # Calculate `post_sd`
    post_sd = np.sqrt(np.maximum(0, post_mean2 - post_mean**2))

    # Update `post_mean2` and `post_mean`
    post_mean2 = post_mean2 + mu**2 + 2 * mu * post_mean
    post_mean = post_mean + mu

    # Return the results as an instance of PosteriorMeanExpResult
    return PosteriorMeanExp(post_mean, post_mean2, post_sd)


In [5]:
betahat=  np.array([1,2,3,4,5])
sebetahat=np.array([1,0.4,5,1,1])
scale = autoselect_scales_mix_norm ( np.array([1,2,3,4,5]),  np.array([1,0.4,5,1,1]))
location = 0* scale
n=betahat.shape[0]
p= scale.shape[0]
 
log_pi =  np.log( np.full( (n, p), 1/scale.shape[0]))
print(scale)
print(location)
#res = posterior_mean_exp(betahat, sebetahat, log_pi, scale)
#print(res.post_mean)
#print(res.post_mean2)

[0.         0.03827328 0.07654655 0.15309311 0.30618622 0.61237244
 1.22474487 2.44948974 4.89897949 9.79795897]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [42]:
#compute gaussian likelihood

convolved_logpdf.normal <- function(  betahat, sebetahat, location, scale):
  # Calculate the standard deviation
    sd = np.sqrt(se**2 +  scale**2)
    
    # Calculate the log probability density
    logp = norm.logpdf(betahat, loc= location, scale=sd)
    
    # Clamp the log probabilities to the range [-1e4, 1e4]
    logp = np.clip(logp, -1e4, 1e4)
    
    return logp

[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]


writing wpost_def

In [59]:
w=assignment
print(w)
wpost_exp ( x, s, w, scale)
temp_array =   np.zeros ( (betahat.shape[0], scale.shape[0]))
i=1
temp_array[i,] = wpost_exp ( x=betahat[i], s=sebetahat[i], w=w, scale=scale) 
print(temp_array[i,])

[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0.06666667]
[3.53987758e-06 6.61493885e-06 1.06391083e-05 2.86976960e-05
 1.69165759e-04 1.40776600e-03 8.91255655e-03 3.45101678e-02
 8.34205613e-02 1.38160703e-01 1.72844260e-01 1.77093219e-01
 1.57939387e-01 1.28091688e-01 9.74010338e-02]


res.post_mean should be equal to [0.4836384 1.8932834 1.0567097 3.5700976 4.6637965]
res.post_mean2 should be equal to [ 0.5967422  3.7537202  4.3433782 13.8929910 22.8110722]